# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Импортируем библиотеки, функции и данные. Посмотрим на наличие пропусков.

In [89]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt 
import itertools
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from tqdm import tqdm
import seaborn as sns
from IPython.display import display
from scipy import stats

In [90]:
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [91]:
print(df_0.info())
print(df_1.info())
print(df_2.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory 

Наиличие дубликатов:

In [92]:
print(df_0.duplicated().sum())
print(df_1.duplicated().sum())
print(df_2.duplicated().sum())

0
0
0


И сами данные:

In [93]:
display(df_0.head())
display(df_1.head())
display(df_2.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Посмотрим на распределения значений различных столбцов в данных

In [94]:
print(df_0.describe())
print(df_1.describe())
print(df_2.describe())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%       

Посмотрим на корреляцию значений в столбцах:

In [95]:
print(df_0.corr())
print(df_1.corr())
print(df_2.corr())

               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000
               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000
               f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000


### Вывод

Данные загружены и изучены. Аномалий нет, пропусков и дубликатов тоже. Формат всех данных - float64. Есть небольшие выбросы в df_1['f2'],df_2['f0', 'f1'] и df_3['f2']. Ничего с ними делать не будем. Но будем иметь это ввиду. Корреляция просматривается не везде, однако стоит обратить внимание на корреляцию в df_2 - там данные из столбца f2 почти полностью коррелируют с данным из столбца с запасами сырья. Странное явление. 

## Обучение и проверка модели

Проведем разделение признаков и целей, разделим данные на выборки, обучим модедь и посчитаем среднюю квадратичную ошибку.

In [96]:
# делим на признаки и целевой параметр
df_0_features = df_0.drop(["id","product"], axis = 1)
df_0_target = df_0["product"]

# разделим данные на выборки:
df_0_features_train, df_0_features_valid, df_0_target_train, df_0_target_valid = train_test_split(
    df_0_features, df_0_target, test_size=0.25, random_state=12345)

# инициилизируем модель:
model_df_0 = LinearRegression()

# обучим модель на данных df_1
model_df_0.fit(df_0_features_train, df_0_target_train)
predict_df_0 = model_df_0.predict(df_0_features_valid)
predict_df_0_mean = predict_df_0.mean()

# найдем среднюю квадратичную ошибку:
rmse_df_0 = mean_squared_error(df_0_target_valid, predict_df_0) ** 0.5

print('Средний запас предсказанного сырья:', predict_df_0_mean)
print('RMSE модели:', rmse_df_0)

Средний запас предсказанного сырья: 92.59256778438038
RMSE модели: 37.5794217150813


Проделаем те же операции с оставшимися наборами данных:


In [97]:
# делим на признаки и целевой параметр
df_1_features = df_1.drop(['id','product'], axis = 1)
df_1_target = df_1['product']

# разделим данные на выборки:
df_1_features_train, df_1_features_valid, df_1_target_train, df_1_target_valid = train_test_split(
    df_1_features, df_1_target, test_size=0.25, random_state=12345)

# инициилизируем модель:
model_df_1 = LinearRegression()

# обучим модель на данных df_1
model_df_1.fit(df_1_features_train, df_1_target_train)
predict_df_1 = model_df_1.predict(df_1_features_valid)
predict_df_1_mean = predict_df_1.mean()

# найдем среднюю квадратичную ошибку:
rmse_df_1 = mean_squared_error(df_1_target_valid, predict_df_1) ** 0.5

print('Средний запас предсказанного сырья:', predict_df_1_mean)
print('RMSE модели:', rmse_df_1)

Средний запас предсказанного сырья: 68.728546895446
RMSE модели: 0.893099286775616


In [98]:
# делим на признаки и целевой параметр
df_2_features = df_2.drop(['id','product'], axis = 1)
df_2_target = df_2['product']

# разделим данные на выборки:
df_2_features_train, df_2_features_valid, df_2_target_train, df_2_target_valid = train_test_split(
    df_2_features, df_2_target, test_size=0.25, random_state=12345)

# инициилизируем модель:
model_df_2 = LinearRegression()

# обучим модель на данных df_1
model_df_2.fit(df_2_features_train, df_2_target_train)
predict_df_2 = model_df_2.predict(df_2_features_valid)
predict_df_2_mean = predict_df_2.mean()

# найдем среднюю квадратичную ошибку:
rmse_df_2 = mean_squared_error(df_2_target_valid, predict_df_2) ** 0.5

print('Средний запас предсказанного сырья:', predict_df_2_mean)
print('RMSE модели:', rmse_df_2)

Средний запас предсказанного сырья: 94.96504596800489
RMSE модели: 40.02970873393434


Здесь я понимаю что возможно стоит завернуть расчеты в функцию, поскольку действия повторяются, но с этим у меня проблемы. Мне нужна помощь - что и как лучше поместить в функцию, что дать на вход и что она должна вернуть. Нужна помощь)

### Вывод

Средний запас предсказанного сырья во втором наборе данных немного ниже - 69, в сравнении с остальными - 93 и 95 соответственно. В то же время RMSE на втором датасете намного ниже - 0.9, в сравнении с теми же 38 и 40. Вывод - средний набор данных сильно отличается. Мы помним там есть аномальная корреляция...

## Подготовка к расчёту прибыли

Нам необходимо рассчитать прибыльность разведки. Напомним данные, которые у нас есть:
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки
- Бюджет на разработку скважин в регионе — 10 млрд рублей
- При нынешних ценах один баррель сырья приносит 450 рублей дохода
- Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%

Создадим переменные и посчитаем безубыточность разработки новой скважины, сравним со средним запасом в регионе.

In [99]:
total_money = 10**10
top_spot = 200
max_spot = 500
income = 4.5*10**5

In [100]:
min_prod = total_money / (top_spot * income)
min_prod

111.11111111111111

### Вывод

Минимальный объем для безубыточной добычи составляет 111.11 (тыс.баррелей). Что несомненно выше средних предсказаний, даже в самом прибыльном регионе - 95 (тыс.баррелей)

## Расчёт прибыли и рисков 

### Расчет прибыли

In [101]:
# функция для подсчета прибыли(подсмотрел в тренажере):

def profit(predictions, target, top_spot):    
    predictions = pd.Series(predictions, index=target.index)
    predictions_best = predictions.sort_values(ascending=False)
    target_best = target[predictions_best.index][:top_spot]
    return sum(target_best)*income - total_money

Используем функцию для подсчета прибыли на всех площадках


In [102]:
profit_0 = profit(predict_df_0, df_0_target_valid, top_spot)
profit_1 = profit(predict_df_1, df_1_target_valid, top_spot)
profit_2 = profit(predict_df_2, df_2_target_valid, top_spot)

In [103]:
print("Потенциальная прибыль в регионе 0:", profit_0/1000000000, 'млрд. руб')
print("Потенциальная прибыль в регионе 1:", profit_1/1000000000, 'млрд. руб')
print("Потенциальная прибыль в регионе 2:", profit_2/1000000000, 'млрд. руб')

Потенциальная прибыль в регионе 0: 3.3208260431398546 млрд. руб
Потенциальная прибыль в регионе 1: 2.415086696681551 млрд. руб
Потенциальная прибыль в регионе 2: 2.7103499635998363 млрд. руб


### Расчет рисков

Напишем функцию для bootstrap, чтобы улучшить предсказания, и посчитаем риски

In [104]:
#функция для bootstrap (подсмотрел в slack)
def bootstrap(predictions, target, profits):
    state = np.random.RandomState(0)
    target = pd.Series(target)
    values = []
 
    for i in range(1000):
        target_subsamples = target.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
        predictions_subsamples = predictions[target_subsamples.index]
        values.append(profit(predictions_subsamples, target_subsamples, top_spot))
 
    values = pd.Series(values)
    mean_profit = values.mean()
    loss = np.mean(values < 0)
    
    #доверительный интервал:
    profits = pd.Series(profits)
    conf_int_left = profits.quantile(0.025)
    conf_int_right = profits.quantile(0.975)
    
    return mean_profit, loss, conf_int_left, conf_int_right

In [105]:
mean_profit_0, risk_0, conf_int_left_0, conf_int_right_0 = bootstrap(predict_df_0, df_0_target_valid, profit_0)
mean_profit_1, risk_1, conf_int_left_1, conf_int_right_1 = bootstrap(predict_df_1, df_1_target_valid, profit_1)
mean_profit_2, risk_2, conf_int_left_2, conf_int_right_2 = bootstrap(predict_df_2, df_2_target_valid, profit_2)

In [106]:
print('Седняя прибыль в регионе 0:', mean_profit_0/1000000, 'млн. руб')
print('Риск в регионе 0:', risk_0*100,'%')
print('Доверительный интервал в регионе 0 лежит между', conf_int_left_0, conf_int_right_0)

Седняя прибыль в регионе 0: 410.30332713249203 млн. руб
Риск в регионе 0: 7.7 %
Доверительный интервал в регионе 0 лежит между 3320826043.1398544 3320826043.1398544


In [107]:
print('Седняя прибыль в регионе 1:', mean_profit_1/1000000, 'млн. руб')
print('Риск в регионе 1:', risk_1*100,'%')
print('Доверительный интервал в регионе 1 лежит между', conf_int_left_1, conf_int_right_1)

Седняя прибыль в регионе 1: 507.5524766187899 млн. руб
Риск в регионе 1: 0.6 %
Доверительный интервал в регионе 1 лежит между 2415086696.681551 2415086696.681551


In [108]:
print('Седняя прибыль в регионе 2:', mean_profit_2/1000000, 'млн. руб')
print('Риск в регионе 2:', risk_2*100,'%')
print('Доверительный интервал в регионе 2 лежит между', conf_int_left_2, conf_int_right_2)

Седняя прибыль в регионе 2: 412.2633519279241 млн. руб
Риск в регионе 2: 7.5 %
Доверительный интервал в регионе 2 лежит между 2710349963.5998363 2710349963.5998363


### Вывод

Учитывая полученные данные, можно сделать вывод, что наименьший риск разработки нефтедобычи мы имеем в регионе 1 - на уровне 0.6%. Там же наибольшая средняя потенциальная прибыль - 507 млн рублей.

## Общий вывод

1. В регионах 0 и 2 достаточно высокий средний показатель запаса предсказанного сырья. Однако RMSE для данных регионов также высок (37 и 40 соответственно). Это свидетельствует о неоднозначности показателя, неточности модели регрессии.

2. В регионе 1 средний показатель запаса предсказанного сырья составляет 68.7 (ниже остальных регионов). Однако RMSE в данном регионе также мал (0.9). Это говорит о точности предсказаний и качестве построенной модели.

3. Минимальный объем для безубыточной добычи составляет 111.11 (тыс.баррелей). Что несомненно выше средних предсказаний, даже в самом прибыльном регионе - 95 (тыс.баррелей)

4. Оценка прибыли наибольшее значение имеет в регионе 1 - 507 млн рублей, против 410 и 412 млн рублей в 0 и 2 регионе соответственно. 

5. Риск убытков минимален для региона 1 и составляет 0.6%, против 7.7% и 7.5% в 0 и 2 регионах соответственно.

6. Рекомендовать регион 1 для разработки. 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: модели обучены и проверены
    - [ ]  Данные корректно разбиты на обучающую и валидационную выборки
    - [ ]  Модели обучены, предсказания сделаны
    - [ ]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [ ]  На экране напечатаны результаты
    - [ ]  Сделаны выводы
- [ ]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [ ]  Для всех ключевых значений созданы константы Python
    - [ ]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [ ]  По предыдущему пункту сделаны выводы
    - [ ]  Написана функция расчёта прибыли
- [ ]  Выполнен шаг 4: посчитаны риски и прибыль
    - [ ]  Проведена процедура *Bootstrap*
    - [ ]  Все параметры бутстрепа соответствуют условию
    - [ ]  Найдены все нужные величины
    - [ ]  Предложен регион для разработки месторождения
    - [ ]  Выбор региона обоснован